In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import json

In [4]:
net = cv2.dnn.readNetFromDarknet("yolov3_Defect.cfg",r"yolov3_Defect_2000.weights")
classes = ['Defect']

In [7]:
defect_type = []
img_name_type = []
defect_box = []
img_name_box = []

for sr in range(1,1101) : 

    if(sr!=7) : 
        sr_s = str(sr)
        sr_l = len(sr_s)
        name = "0"*(6-sr_l) + sr_s + ".png"
        img_name_type.append(name)
        img = Image.open(f'Test\\{name}')
        img = np.array(img, dtype=np.uint8)
        hight,width,_ = img.shape
        blob = cv2.dnn.blobFromImage(img, 1/255,(416,416),(0,0,0),swapRB = True,crop= False)

        net.setInput(blob)

        output_layers_name = net.getUnconnectedOutLayersNames()

        layerOutputs = net.forward(output_layers_name)

        boxes =[]
        confidences = []
        class_ids = []

        for output in layerOutputs:
            for detection in output:
                score = detection[5:]
                class_id = np.argmax(score)
                confidence = score[class_id]
                if confidence > 0.7:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * hight)
                    w = int(detection[2] * width)
                    h = int(detection[3]* hight)
                    x = int(center_x - w/2)
                    y = int(center_y - h/2)
                    boxes.append([x,y,w,h])
                    confidences.append((float(confidence)))
                    class_ids.append(class_id)


        indexes = cv2.dnn.NMSBoxes(boxes,confidences,.5,.4)

        boxes =[]
        confidences = []
        class_ids = []

        for output in layerOutputs:
            for detection in output:
                score = detection[5:]
                class_id = np.argmax(score)
                confidence = score[class_id]
                if confidence > 0.00:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * hight)
                    w = int(detection[2] * width)
                    h = int(detection[3]* hight)
                    x = int(center_x - w/2)
                    y = int(center_y - h/2)
                    boxes.append([x,y,w,h])
                    confidences.append((float(confidence)))
                    class_ids.append(class_id)
        #print(confidences)
        indexes = cv2.dnn.NMSBoxes(boxes,confidences,.1,.1)
        #print(indexes)
        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0,255,size =(len(boxes),3))
        if  len(indexes)>0:
            defect_type.append(1)
            for i in indexes.flatten():
                img_name_box.append(name)
                #print(boxes[i])
                x,y,w,h = boxes[i]
                defect_box.append([(x+w/2)/width,(y+h/2)/hight,w/width,h/hight])
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i],2))
                color = colors[i]
                cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
                cv2.putText(img,label + " " + confidence, (x,y+400),font,2,color,2)
            im = Image.fromarray(img)
            im.save(f"C:\\Users\\iammr\\OneDrive\\Desktop\\Python Files\\CV Hackathon\\Test_box\\{name}") 
        else : defect_type.append(0) 
        
with open('defect_type.json', 'w') as f: json.dump(defect_type, f) 
with open('img_name_type.json', 'w') as f: json.dump(img_name_type, f) 
with open('defect_box.json', 'w') as f: json.dump(defect_box, f) 
with open('img_name_box.json', 'w') as f: json.dump(img_name_box, f) 